# Generate Depth Map
---

In [0]:
import torch
from IPython.display import Image, clear_output 
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

PyTorch 1.5.0+cu101 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!git clone https://github.com/genigarus/DenseDepth.git

In [0]:
from PIL import Image
from io import BytesIO

def load_zip_images(image_files):
    loaded_images = []
    for file in image_files:
        x = np.clip(np.asarray(Image.open(BytesIO(data[ file ])), dtype=float) / 255, 0, 1)
        loaded_images.append(x)
    return np.stack(loaded_images, axis=0)

def loadZipToMem(zip_file):
    # Load zip file into memory
    print('Loading dataset zip file...', end='')
    from zipfile import ZipFile
    input_zip = ZipFile(zip_file, 'r')
    data = {name: input_zip.read(name) for name in input_zip.namelist()}
    input_zip.close()
    
    return data

def save_zip_images(filename, outputs):
    image = outputs.reshape(outputs.shape[0],outputs.shape[1])
    fig = plt.figure(figsize=(1.60, 1.60))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(image, aspect='equal')
    plt.savefig(filename, dpi=100)
    ax.cla()
    ax.clear()
    plt.close(fig)

data = loadZipToMem('./gdrive/My Drive/EVA4/DepthMaskDataset/depth_mask_custom_dataset.zip')

custom_data = list((row.split('\t') for row in (data['data_label.txt']).decode("utf-8").split('\n') if len(row) > 0))
fg_bg_list = []
for fg_bg_img_row in custom_data:
  fg_bg_list.append(fg_bg_img_row[0])


Loading dataset zip file...

In [0]:
os.chdir('./DenseDepth')

In [0]:
!mkdir NYU/weights

mkdir: cannot create directory ‘NYU/weights’: File exists


In [0]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O ./NYU/weights/nyu.h5  #0.01GB

--2020-05-10 15:44:54--  https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.84.74
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.84.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172897376 (165M) [application/h5]
Saving to: ‘./NYU/weights/nyu.h5’

./NYU/weights/nyu.h 100%[===================>] 164.89M  22.6MB/s    in 8.3s    

2020-05-10 15:45:03 (19.8 MB/s) - ‘./NYU/weights/nyu.h5’ saved [172897376/172897376]



In [0]:
import math, random, glob, os
# from random import random
# from random import seed

import cv2
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm

from CreateDepthMaskNYU import CreateDepthMaskNYU
import matplotlib.pyplot as plt
from PIL import Image

import zipfile
import gc

Using TensorFlow backend.


In [0]:
model_path = r'./NYU/weights/nyu.h5'

In [0]:
import glob
import os, time

import matplotlib.pyplot as plt
from PIL import Image
import NYU
from NYU import BilinearUpSampling2D, predict, scale_up, display_images, load_images


class CreateDepthMask:

    def __init__(self, model_path):
        self.model = None

        if os.path.isfile(model_path):
            print('Loading model...')

            # Custom object needed for inference and training
            custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': None}

            # Load model into GPU / CPU
            self.model = NYU.load_model(model_path, custom_objects=custom_objects, compile=False)

            print('\nModel loaded ({0}).'.format(model_path))

    def get_depth_map_from_zip(self, image_list, scale, batch_size=1):

        if self.model is not None:
            # Input images
            t1 = time.time()
            inputs = load_zip_images(image_list)
            t2 = time.time()
            print(f'\nLoaded ({inputs.shape[0]}) images of size {inputs.shape[1:]} in {t2-t1} sec.')

            if scale > 1:
                inputs = scale_up(scale, inputs)

            # Compute results
            t1 = time.time()
            outputs = predict(self.model, inputs, batch_size)
            t2 = time.time()
            print(f'Actual prediction time: {t2-t1} sec.')
            return outputs


In [0]:
depth_model = CreateDepthMask(model_path) #0.72GB

Loading model...

Model loaded (./NYU/weights/nyu.h5).


In [0]:
os.chdir('../gdrive/My Drive/EVA4/DepthMaskDataset')

In [0]:
import zipfile
import gc
import numpy as np

zip_file_name = "DepthMapData.zip"
bs = 200*20
batch_size = 32
for i in tqdm(range(0, 100)):
  print(f"\nIteration{i+1}")

  # Input images
  print("Predicting....")
  outputs = depth_model.get_depth_map_from_zip(fg_bg_list[bs*i: (bs*i)+bs], 1, batch_size)

  print("Saving...")
  zf1 = zipfile.ZipFile(zip_file_name, mode='a', compression=zipfile.ZIP_STORED)
  label_info = open('data_label.txt', 'a+')
  for itr in range(outputs.shape[0]):
    filename = f"/{fg_bg_list[(bs*i)+itr].split('/')[-1].split('.')[0]}_depthmap_{fg_bg_list[(bs*i)+itr].split('/')[-1].split('.')[0].split('_')[-1]}.jpg"
    label_info.write(f'Batch {i}\tdata/fg_bg_depth_map/batch_{i:03d}/{filename}\n')
    save_zip_images("depth_map.jpg", outputs[itr])
    zf1.write("depth_map.jpg", f'data/fg_bg_depth_map/batch_{i:03d}/{filename}')
  label_info.close()
  zf1.close()

  np.savez_compressed(f'data/fg_bg_depth/depthmap_output_batch{i+1}.npz', output=outputs)

  del outputs
  del label_info
  del zf1
  gc.collect()
  



Iteration1
Predicting....

Loaded (inputs.shape[0]) images of size (160, 160, 3) in 12.75155234336853 sec.
Actual prediction time: 120.39214968681335 sec.
Saving...

Iteration2
Predicting....

Loaded (inputs.shape[0]) images of size (160, 160, 3) in 11.079334020614624 sec.
Actual prediction time: 102.04290843009949 sec.
Saving...

Iteration3
Predicting....

Loaded (inputs.shape[0]) images of size (160, 160, 3) in 13.701427698135376 sec.
Actual prediction time: 101.24023127555847 sec.
Saving...

Iteration4
Predicting....

Loaded (inputs.shape[0]) images of size (160, 160, 3) in 9.89923357963562 sec.
Actual prediction time: 102.24694514274597 sec.
Saving...

Iteration5
Predicting....

Loaded (inputs.shape[0]) images of size (160, 160, 3) in 9.71603536605835 sec.
Actual prediction time: 101.81047201156616 sec.
Saving...

Iteration6
Predicting....

Loaded (inputs.shape[0]) images of size (160, 160, 3) in 12.5301513671875 sec.
Actual prediction time: 100.87504434585571 sec.
Saving...

Iter

In [0]:
import zipfile
import gc
import numpy as np

#zip_file_name = "DepthMapData.zip"
bs = 200*20
batch_size = 32
for i in tqdm(range(10, 100)):
  print(f"\nIteration{i+1} - Images [{bs*i}: {(bs*i)+bs}]")

  # Input images
  print("Predicting....")
  outputs = depth_model.get_depth_map_from_zip(fg_bg_list[bs*i: (bs*i)+bs], 1, batch_size)

  print("Saving...")
  zf1 = zipfile.ZipFile(zip_file_name, mode='a', compression=zipfile.ZIP_STORED)
  label_info = open('data_label.txt', 'a+')
  for itr in range(outputs.shape[0]):
    filename = f"/{fg_bg_list[(bs*i)+itr].split('/')[-1].split('.')[0]}_depthmap_{fg_bg_list[(bs*i)+itr].split('/')[-1].split('.')[0].split('_')[-1]}.jpg"
    label_info.write(f'Batch {i}\tdata/fg_bg_depth_map/batch_{i:03d}/{filename}\n')
    save_zip_images("depth_map.jpg", outputs[itr])
    zf1.write("depth_map.jpg", f'data/fg_bg_depth_map/batch_{i:03d}/{filename}')
  label_info.close()
  zf1.close()

  np.savez_compressed(f'data/fg_bg_depth/depthmap_output_batch{i+1}.npz', output=outputs)

  del outputs
  del label_info
  del zf1
  gc.collect()
  



Iteration11 - Images [40000: 44000]
Predicting....

Loaded (4000) images of size (160, 160, 3) in 13.600987672805786 sec.
Actual prediction time: 133.02228021621704 sec.
Saving...

Iteration12 - Images [44000: 48000]
Predicting....

Loaded (4000) images of size (160, 160, 3) in 10.473581075668335 sec.
Actual prediction time: 115.40894675254822 sec.
Saving...

Iteration13 - Images [48000: 52000]
Predicting....

Loaded (4000) images of size (160, 160, 3) in 10.163222789764404 sec.
Actual prediction time: 113.36479067802429 sec.
Saving...

Iteration14 - Images [52000: 56000]
Predicting....

Loaded (4000) images of size (160, 160, 3) in 9.976438045501709 sec.
Actual prediction time: 113.50731253623962 sec.
Saving...

Iteration15 - Images [56000: 60000]
Predicting....

Loaded (4000) images of size (160, 160, 3) in 10.180702686309814 sec.
Actual prediction time: 113.20022010803223 sec.
Saving...

Iteration16 - Images [60000: 64000]
Predicting....

Loaded (4000) images of size (160, 160, 3) 